<a href="https://colab.research.google.com/github/NolanSmyth/deepMIDI/blob/master/midiMusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependences

In [9]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 

import numpy as np
import os
import glob
from music21 import *

import urllib.request


In [ ]:
with urllib.request.urlopen('http://www.python.org/') as f:
  print(f.read(300))

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
notes = []
midfs = glob.glob('/content/drive/My Drive/midi/*.mid')
for midf in midfs:
  midi = converter.parse(midf)
notes_to_parse = midi.flat.notes
for element in notes_to_parse:
  if isinstance(element, note.Note):
    notes.append(str(element.pitch))
  elif isinstance(element, note.Chord):
    notes.append('.'.join(str(n) for n in element.normalOrder))

In [76]:
sequence_length = 10

# get names of all pitches used
pitch_names = sorted(set(notes))

# create a dictionary mapping each name to a unique integer
note_to_int = dict((note, i) for i, note in enumerate(pitch_names))

# create input and output sequences to feed into neural net
net_in = []
net_out = []
for i in range(0, len(notes) - sequence_length):
  sequence_in = notes[i:i + sequence_length]
  sequence_out = notes[i + sequence_length]
  net_in.append([note_to_int[char] for char in sequence_in])
  net_out.append([note_to_int[sequence_out]])

In [77]:
n_sequences = len(net_in)
net_in = np.reshape(net_in, (n_sequences, sequence_length, 1))